<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Constants" data-toc-modified-id="Constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Constants</a></span></li><li><span><a href="#Utils" data-toc-modified-id="Utils-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utils</a></span></li><li><span><a href="#NetworkX-tutorials" data-toc-modified-id="NetworkX-tutorials-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NetworkX tutorials</a></span></li><li><span><a href="#Method-1:-Randomly-generate-triangles-and-connect-them-together" data-toc-modified-id="Method-1:-Randomly-generate-triangles-and-connect-them-together-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Method 1: Randomly generate triangles and connect them together</a></span></li><li><span><a href="#Method-2:-Complete-Graph-with-Random-Weights-Adjusted" data-toc-modified-id="Method-2:-Complete-Graph-with-Random-Weights-Adjusted-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Method 2: Complete Graph with Random Weights Adjusted</a></span></li><li><span><a href="#Method-3:-Generate-Random-Coordinates-Within-Bounds" data-toc-modified-id="Method-3:-Generate-Random-Coordinates-Within-Bounds-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Method 3: Generate Random Coordinates Within Bounds</a></span></li><li><span><a href="#Formatting-Inputs,-Outputs" data-toc-modified-id="Formatting-Inputs,-Outputs-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Formatting Inputs, Outputs</a></span></li></ul></div>

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random as rd
import csv
import numpy as np
import math
import pickle

# Constants

In [ ]:
# Constants
MAX = 1e9  # maximum value of edge weight
MIN = 1  # minimum value of edge weight (not real min because we account for FACTOR)
NUM_DEC = 5  # number of decimal numbers for rounding
FACTOR = 0.9 # FACTOR of edge weight
NUM_EDGE_FACTOR = 0.1
START_FACTOR = 0.5
MIN_P = 0.55 # minimum percentage
MAX_P = 0.95 # maximum percentage
SMALL = 48   # small input
MEDIUM = 98 # medium input
LARGE = 198  # large input
TEST_SIZE = 10  # use this to change the size of test cases

# maximum number of TA's / homes
SMALL_TA = 23
MEDIUM_TA = 48
LARGE_TA = 98

# Utils

In [ ]:
def write_to_file(file, string, append=False):
    if append:
        mode = 'a'
    else:
        mode = 'w'
    with open(file, mode) as f:
        f.write(string)


def write_data_to_file(file, data, separator, append=False):
    if append:
        mode = 'a'
    else:
        mode = 'w'
    with open(file, mode) as f:
        for item in data:
            f.write(f'{item}{separator}')
            
def read_file_to_list(file, length):
    lst = []
    with open(file, 'r') as f:
        lst = f.read().splitlines()
    return lst[:length]
            
def print_edge_weights(g, n=5):
    for edge, data in g.edges.items():
        if n <= 0:
            break
        n-=1
        print(edge, data)
        
def convert(lst):
    return ' '.join(lst)

def fix_triangle_inequalities(g):
    done = False
    count = 0
    num_fixes = 0
    while not done:
        done = True
        rs = nx.floyd_warshall(g, weight='weight')
        print("iteration ", count)
        for edge, data in g.edges.items():
            if data['weight'] > rs[edge[0]][edge[1]]:
#                 print('change ' + str(edge) + " " + str(data['weight']) + " to " + str(rs[edge[0]][edge[1]]))
                data['weight'] = rs[edge[0]][edge[1]]
                done = False
                num_fixes += 1
#                 break # with break will lead to less fixes but takes more time to update
        count += 1
    print("total fixes: ", num_fixes)
    
def save_graph(g, filename):
    pickle.dump(g, open(filename, "wb"))

def load_graph(filename):
    return pickle.load( open(filename, "rb"))

# NetworkX tutorials

In [ ]:
# G = nx.Graph([(1,1), (2,2), (1,2), (1,2)])
G = nx.Graph()
G.add_node(1)
G.add_node(2)
G.add_edge(1, 2)
G.add_edge(2, 2)
A = nx.adjacency_matrix(G)
print(A.todense())

In [ ]:
G = nx.Graph()
# H = nx.path_graph(2)
# G.add_nodes_from(H)
G = nx.petersen_graph()
# pos = nx.nx_agraph.graphviz_layout(G)
# nx.draw(G, pos=pos)
# plt.show()
# plt.subplot(121)

# nx.draw(G, with_labels=True, font_weight='bold')
# plt.subplot(122)

# nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')


In [ ]:
G1 = nx.complete_graph(3)
# plt.subplot(121)
# nx.draw(G1, node_color='blue', with_labels=True)
print(list(G1.nodes()))
print(list(G1.edges()))

G2 = nx.complete_graph(range(3,6))
# plt.subplot(122)
# nx.draw(G2, node_color='red', with_labels=True)

G3 = nx.union(G1, G2)
G3.add_edge(2, 4, weight=4.7)
print(G3[4][2])
nx.draw(G3, node_color='red', with_labels=True)
print(list(G.nodes()).pop())

# Method 1: Randomly generate triangles and connect them together

In [ ]:
def add_weights(g):
    w1 = rd.uniform(MIN, MAX)
    w2 = rd.uniform(MIN_P * w1, MAX_P * w1)
    w3 = rd.uniform(MIN_P * w1, MAX_P * w1)
    ws = [w1, w2, w3]
    for _, data in g.edges.items():
        data['weight'] = round(ws.pop(), NUM_DEC)
    
def connect(G, g):
    Gnodes = list(G.nodes)
    gnodes = list(g.nodes)
    G = nx.union(G, g)
    for n in gnodes:
        idx = rd.randrange(0, len(Gnodes))
        a = Gnodes.pop(idx)
        try:
            w = nx.shortest_path_length(G, source=a, target=n) * FACTOR
        except nx.NetworkXNoPath:
            ws = [i for _, _, i in g.edges.data('weight')]
            w = rd.uniform(min(ws), max(ws)) * FACTOR
        G.add_edge(a, n, weight=round(w, NUM_DEC))
    return G
        
def gen_graph(n):
    G = nx.Graph()
    while n:
        a = n.pop()
        b = n.pop()
        c = n.pop()
        nodes = [a, b, c]
        g = nx.complete_graph(nodes)
        add_weights(g)
        if len(G.nodes) == 0:
            G = g
        else:
            G = connect(G, g)
    return G

In [ ]:
def convertTo
try:
    nx.shortest_path_length(G, source=2, target=3)
except nx.NetworkXNoPath:
    print("No path")
# for e in G.edges.items():
#     print(e)
nx.draw(G, node_color='red', with_labels=True)
A = nx.adjacency_matrix(G).toarray()
B = [[None] * len(A[0])] * len(A)
for i in range(len(A)):
    for j in range(len(A[0])):
        if A[i][j] == 0.0:
            B[i][j] = 'x'
        else:
            B[i][j] = A[i][j]
print(B)

# Method 2: Complete Graph with Random Weights Adjusted

Generate a complete graph with random weights associated each edge. Use floyd_warshall to produce the 
all-pairs shortest path distances. For each edge, set the length of the edge to be the length of the shortest distance between those two vertices.

Drawbacks: After adjusting the weights, some points become lying along the way between two points as their middle points. It also requires more time to generate a valid graph.

In [ ]:
# Generate 
G_final = nx.complete_graph(LARGE)
nx.draw(G_final, node_color='yellow', with_labels=True)

# print(list(G1.nodes()))
# print(list(G1.edges()))

In [ ]:
def add_weights2(g, lower, upper):
    """Update the weight of each edge to be a random weight within the bounds."""
    for edge, data in g.edges.items():
        w = rd.uniform(lower, upper)
        data['weight'] = round(w, NUM_DEC)
#         print(edge, data)        

In [ ]:
# dict1 = nx.floyd_warshall(G_final, weight='weight')
# plt.subplot(121)
# nx.draw(G_final, node_color='yellow', with_labels=True)
# print(list(G_final.nodes()))
# print(list(G_final.edges().items()))
# dict1 = nx.all_pairs_shortest_path_length(G_final)

In [ ]:
def print_shortest_pair_distance(g):
    rs = nx.floyd_warshall(g)
    for i in rs:
        for j in rs[i]:
            print ("%d\t%d\t%f" % (i, j, rs[i][j]))

Generate random weight for each edge.

In [ ]:
# G_final[2][3]['weight'] = 78.73409
add_weights2(G_final, MIN, MAX)
print_edge_weights(G_final)  

Fix the weight to satisfy the trangle inequalities.

In [ ]:
fix_triangle_inequalities(G_final)
print_edge_weights(G_final)  

# Method 3: Generate Random Coordinates Within Bounds

In this method, we treat the locations as a point on a 2D plane with x, y coordinates. If we connect any three locations as a triangle and set the distance between any two locations as the edge weight, then the edge weights are guaranteed to satisfy the triangle inequality because on a plane the shortest path between two points is the edge directly connect them.

Genearte random points with coordinates (x, y), where $0 < x < 10^9$ and $0 < y < 10^9$, so the maximum possible weight is $\sqrt{(10^9)^2 + (10^9)^2} = \sqrt{2 \times 10^9} < 2 \times 10^9$.

In [ ]:
def gen_rand_loc_in_range(num: int, xmin: float, xmax: float, ymin: float, ymax: float) -> list:
    """Generate a list of random locations in the provided ranges.
    
    Each location is a tupple of x, y coordinates, (x_i, y_i).
    * xmin <= x_i <= xmax
    * ymin <= y_i <= ymax
    
    Parameters:
        num: int
            The number of locations
        xmin: float
            The minimum value of x coordinate
        xmax: float
            The maximum value of x coordinate
        ymin: float
            The minimum value of y coordinate
        ymax: float
            The maximum value of y coordinate
    
    Returns
        list
            A list of locations
    """
    
    locs = []
    for i in range(num):
        x = round(rd.uniform(xmin, xmax), NUM_DEC)
        y = round(rd.uniform(ymin, ymax), NUM_DEC)
        locs.append((x, y))
    return locs

def euclidean_dist(p1, p2):
    """Calculate the Euclidean distance between two points"""
    
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def plot_locs(locations):
    """coords: list of tuples (x, y)"""
    
    coords = np.array(locations)
    x, y = coords.T
    plt.scatter(x,y)
    plt.show()
    
def find_centroid(locations):
    """Find the centroid of the locations"""
    
    maximum_dist = 0
    centroid = locations[0]
    for loc1 in locations:
        for loc2 in locations:
            dist = euclidean_dist(loc1, loc2)
        if (dist > maximum_dist):
            maximum_dist = dist
            centroid = loc1
    return centroid
    
def make_cluster(locations, names):
    """Make a cluster of homes from provided locations and names.
    
    Find the location that closest to other locations, and it is
    the centroid of this cluster. Connect all the homes to the centroid.
    
    loc_name: [coords] => [name]
    """
    
    centroid = find_centroid(locations)
    loc_name = {}
    for i in range(len(locations)):
        loc_name[locations[i]] = names[i]
    g = nx.Graph()
    for item in loc_name.items():
        if item[0] == centroid:
            g.add_node(item[1], coords=item[0], home=0)
        else:
            g.add_node(item[1], coords=item[0], home=1)
    for item in loc_name.items():
        if (item[1] != loc_name[centroid]):
            g.add_edge(item[1], loc_name[centroid], weight=round(euclidean_dist(item[0], centroid), NUM_DEC))
    return (g, loc_name[centroid], centroid)

def make_rand_graph(locations, names):
    g = nx.complete_graph(len(locations))
    keys = list(g.nodes())
    mapping = {}
    for i in range(len(keys)):
        mapping[keys[i]] = names[i]
    g = nx.relabel_nodes(g, mapping)
    nodes = g.nodes()
    loc_name = {}
    for i in range(len(locations)):
        loc_name[locations[i]] = names[i]
    for item in loc_name.items():
        nodes[item[1]]['coords'] = item[0]
    edges = g.edges()
    print(len(edges))
    count = 0
    while count <= len(edges) * NUM_EDGE_FACTOR:
        idx1 = rd.randrange(0, len(locations))
        idx2 = rd.randrange(0, len(locations))
        while idx2 == idx1:
            idx2 = rd.randrange(0, len(locations))
        loc1 = loc_name[locations[idx1]]
        loc2 = loc_name[locations[idx2]]
        try:
            count += 1
            g.remove_edge(loc1, loc2)
        except:
            count -= 1
            continue
        
    
    for e in list(g.edges()):
        edges[e]['weight'] = euclidean_dist(nodes[e[0]]['coords'], nodes[e[1]]['coords'])
    return g

def divide_chunks(home_names):
    n = len(home_names) // 4
    l1 = home_names[:n]
    l2 = home_names[n:2*n]
    l3 = home_names[2*n:3*n]
    l4 = home_names[3*n:]
    return (l1, l2, l3, l4)

def make_clusters(loc_size, home_size):
    if loc_size == SMALL:
        loc_names = read_file_to_list('small_list_names.txt', loc_size)
    elif loc_size == MEDIUM:
        loc_names = read_file_to_list('medium_list_names.txt', loc_size)
    elif loc_size == LARGE:
        loc_names = read_file_to_list('large_list_names.txt', loc_size)
    count = 0
    home_names = []
    while count < home_size:
        idx = rd.randrange(0, len(loc_names))
        home_names.append(loc_names.pop(idx))
        count += 1
    hn1, hn2, hn3, hn4 = divide_chunks(home_names)
    h1 = len(hn1)
    h2 = len(hn2)
    h3 = len(hn3)
    h4 = len(hn4)
    locs1 = gen_rand_loc_in_range(h1, 1, 1/4*MAX, 1, 1/4*MAX)
    locs2 = gen_rand_loc_in_range(h2, 3/4*MAX, MAX, 1, 1/4*MAX)
    locs3 = gen_rand_loc_in_range(h3, 3/4*MAX, MAX, 3/4*MAX, MAX)
    locs4 = gen_rand_loc_in_range(h4, 1, 1/4*MAX, 3/4*MAX, MAX)
    c1, c1_name, c1c = make_cluster(locs1, hn1)
    c2, c2_name, c2c = make_cluster(locs2, hn2)
    c3, c3_name, c3c = make_cluster(locs3, hn3)
    c4, c4_name, c4c = make_cluster(locs4, hn4)
    return (c1, c1_name, c1c, c2, c2_name, c2c, c3, c3_name, c3c, c4, c4_name, c4c, loc_names)
    
def make_graph(loc_size, home_size, start):
    c1, c1_name, c1c, c2, c2_name, c2c, c3, c3_name, c3c, c4, c4_name, c4c, loc_names =\
        make_clusters(loc_size, home_size)
    locs = gen_rand_loc_in_range(len(loc_names), 1, MAX, 1, MAX)
    central_g = make_rand_graph(locs, loc_names)
    central_nodes = central_g.nodes()
    lst_nodes = list(central_nodes)
    for i in range(len(central_nodes)):
        central_nodes[lst_nodes[i]]['home'] = 0
    s_coords = ((c3c[0] + c4c[0])/2, (c3c[1] + c4c[1])/2)
    central_g.add_node(start, coords=s_coords, home=2)
    g = nx.union(central_g, c1)
    g = nx.union(g, c2)
    g = nx.union(g, c3)
    g = nx.union(g, c4)
    g.add_edge(start, c4_name, weight=round(euclidean_dist(s_coords, c4c), NUM_DEC))
    g.add_edge(c4_name, c1_name, weight=round(euclidean_dist(c4c, c1c), NUM_DEC))
    g.add_edge(c1_name, c2_name, weight=round(euclidean_dist(c1c, c2c), NUM_DEC))
    g.add_edge(c2_name, c3_name, weight=round(euclidean_dist(c2c, c3c), NUM_DEC))
    g.add_edge(c3_name, start, weight=round(euclidean_dist(s_coords, c3c), NUM_DEC))
    
    nodes = g.nodes()
    count = 0

    while count < START_FACTOR * len(loc_names):
        idx = rd.randrange(0, len(loc_names))
        count += 1
        g.add_edge(loc_names[idx], start, \
                   weight=round(euclidean_dist(s_coords, nodes[loc_names[idx]]['coords']), NUM_DEC))
    return g

def gen_input(loc_size, home_size, start, filename):
    g = make_graph(loc_size, home_size, start)
    save_graph(g, filename + '.p')
    edges = g.edges()
    lst_edges = list(edges)
    for i in range(len(edges)):
        edges[lst_edges[i]]['weight'] = round(edges[lst_edges[i]]['weight'], NUM_DEC)
    formatInput(g, start, filename + '.in')

In [ ]:
# Test
# locations = gen_rand_loc_in_range(100, 1, 1e6, 1, 1e6)
# plot_locs(locations)
# names = read_file_to_list('large_list_names.txt', 100)
# g, _, _ = make_cluster(locations, names)
# nx.draw(g, node_color='yellow', with_labels=True)
# print(g.edges())
# print(g.nodes().data())
# A = nx.adjacency_matrix(g)
# print(A.todense())

######
# g2 = make_rand_graph(locations, names)
# nx.draw(g2, node_color='red', with_labels=True)
# print(len(g2.edges()))
# print(g2.nodes().data())
# A2 = nx.adjacency_matrix(g2)
# print(A2.todense())
# fix_triangle_inequalities(g2)

######

In [ ]:
gen_input(SMALL, SMALL_TA, 'Soda', '50')
gen_input(MEDIUM, MEDIUM_TA, 'Soda', '100')
gen_input(LARGE, LARGE_TA, 'Soda', '200')

# Formatting Inputs, Outputs

In [ ]:
# f = open('world-cities.csv')
# csv_f = csv.reader(f)
# world_cities = []
# for r in csv_f:
#     name = r[0]
#     if name.isalpha() and name != 'name':
#         world_cities.append(name)
# city_num = len(world_cities)
# small_list = []
# medium_list = []
# large_list = []
# for i in range(SMALL):
#     idx = rd.randrange(0, len(world_cities))
#     small_list.append(world_cities[idx])
# for i in range(MEDIUM):
#     idx = rd.randrange(0, len(world_cities))
#     medium_list.append(world_cities[idx])
# for i in range(LARGE):
#     idx = rd.randrange(0, len(world_cities))
#     large_list.append(world_cities[idx])
# write_data_to_file("small_list_names.txt", small_list, '\n')
# write_data_to_file("medium_list_names.txt", medium_list, '\n')
# write_data_to_file("large_list_names.txt", large_list, '\n')

In [ ]:
small_list = []
medium_list = []
large_list = []
for i in range(SMALL):
    small_list.append('L' + str(i))
for i in range(MEDIUM):
    medium_list.append('L' + str(i))
for i in range(LARGE):
    large_list.append('L' + str(i))
write_data_to_file("small_list_names.txt", small_list, '\n')
write_data_to_file("medium_list_names.txt", medium_list, '\n')
write_data_to_file("large_list_names.txt", large_list, '\n')

In [ ]:
def getFormattedAdjMatrix(G):
    A = nx.adjacency_matrix(G).toarray()
    B = [[0 for i in range(len(A[0]))] for i in range(len(A))]
    for i in range(len(A)):
        for j in range(len(A[0])):
            if A[i][j] == 0.0:
                B[i][j] = 'x'
            else:
                B[i][j] = str(A[i][j])
    return B

In [ ]:
def formatInput(g, start, filename="final_input.txt"):
    # 1. write the # of locations
    write_to_file(filename, str(len(g.nodes())) + '\n')
    homes = []
   
    # construct the list of homes' names
    for n_data in g.nodes().data():
        if n_data[1]['home'] == 1:
            homes.append(n_data[0])
            
    # 2. write the # of homes
    write_to_file(filename, str(len(homes)) + '\n', True)
    
    # 3. write the names of locations
    write_to_file(filename, convert(list(g.nodes())) + '\n', True)
    
    # 4. write the names of homes
    write_to_file(filename, convert(homes) + '\n', True)
    
    # 5. write the name of starting location
    write_to_file(filename, start + '\n', True)
    
    # 5. write the adj matrix
    adjM = getFormattedAdjMatrix(g)
    for i in range(len(adjM)):
        write_to_file(filename, convert(adjM[i]) + '\n', True)

In [ ]:
def formatOutput(car_route, dropoff_names, home_names, filename="final_output.out"):
    """Format and write the output to a file.
    
    Parameters:
        car_route: list of locations'names traveled by car (from start and back to start).
        dropoff_names: list of droff locations' names
        home_names: list of lists of homes' names. each list corresponds to the droff location in dropoff_names
        
        eg.
        ['dropoff_loc1', 'dropoff_loc2']
        [['home1', 'home2'], ['home3']]
        This means we dropp off two TAs at dropoff_loc1 and the TAs walk their homes(home1 and home2).
        Similarilly, we drop off one TA at dropoff_loc2 and one TA walks home3.

    """
    write_to_file(filename, convert(car_route) + '\n')
    write_to_file(filename, str(len(dropoff_names)) + '\n', True)
    for i, loc in enumerate(dropoff_names):
        write_to_file(filename, convert(loc) + ' ', True)
        write_to_file(filename, convert(home_names[i]), True)
        write_to_file(filename, '\n', True)
        
# Test        
# formatOutput(['start', 'dropoff1', 'dropoff2', 'start'], [["dropoff1"], ["droppoff2"]], [["home1", "home2"], ["home3"]])
formatOutput(['Soda'], [['Soda']], [['L20 L7 L2 L32 L9 L29 L34 L42 L45 L13 L17 L46 L31 L12 L14 L22 L23 L4 L33']], '50.out')